We will import the llamaindex components we need

In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding



We will set the llm and embedding model globally.

In [2]:
Settings.llm=Ollama(model="phi3:mini",temperature=0.8,request_timeout=600) #to prevent timeout
Settings.embed_model= OllamaEmbedding(model_name="nomic-embed-text")

Now we will create basic tools for function calling.  The FunctionTool is used for this as given below. The docstring is necessary for the LLM to understand what the tool does.

In [6]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

def subtract(a: float, b: float) -> float:
    """Subtract two numbers and returns the difference"""
    return a-b

sub_tool = FunctionTool.from_defaults(fn=subtract)
add_tool = FunctionTool.from_defaults(fn=add)

Now we intialize the agent using ReActAgent by giving it the functions we created as an array. We can see what is going on by setting verbose=True.

In [7]:
agent = ReActAgent.from_tools([multiply_tool, add_tool,sub_tool], verbose=True)

Now let's ask the question

In [8]:
response = agent.chat("What is (20*2)+4-50? Use a tool to calculate every step.")

Thought: The current language of the user is English. I can answer this question without using a tool, but since you requested using a tool for calculations, I will use the 'multiply' and 'add' tools accordingly.
Action: multiply
Action Input: {'a': 20, 'b': 2}
Observation: 40
Thought: Now that I have calculated the product of 20*2 using the 'multiply' tool, I can proceed to use the 'add' tool for calculating (40 + 4) and then use the 'subtract' tool for subtracting 50 from this sum.
Action: add
Action Input: {'a': 40, 'b': 4}
Observation: 44
Thought: I now have calculated (20*2 + 4) as 44 using the 'add' tool. The final step is to subtract 50 from this sum.
Action: subtract
Action Input: {'a': 44, 'b': 50}
Observation: -6
Thought: I have completed all calculations with the provided tools and can now provide the answer in English as requested by the user.
Answer: The result of (20*2 + 4) - 50 is -6.


Now we can add RAG with our agent. We should import the required dependencie first.